Google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

path_to_drive = "/content/drive/MyDrive/Colab Notebooks"

Mounted at /content/drive


## spec

In [2]:
!nvidia-smi

Mon Sep  6 07:19:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    36W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get GitHub repository files and install python packages

In [3]:
!git clone https://github.com/Team-M1/badwords-classifier-train -b WJ
%cd badwords-classifier-train
!pip install -r requirements.txt

Cloning into 'badwords-classifier-train'...
remote: Enumerating objects: 255, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 255 (delta 141), reused 152 (delta 39), pack-reused 0
Receiving objects: 100% (255/255), 2.95 MiB | 10.31 MiB/s, done.
Resolving deltas: 100% (141/141), done.
/content/badwords-classifier-train
     |████████████████████████████████| 264 kB 4.1 MB/s 
     |████████████████████████████████| 282 kB 45.4 MB/s 
     |████████████████████████████████| 2.8 MB 37.2 MB/s 
     |████████████████████████████████| 919 kB 39.8 MB/s 
     |████████████████████████████████| 119 kB 54.3 MB/s 
     |████████████████████████████████| 243 kB 50.5 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 636 kB 50.2 MB/s 
     |████████████████████████████████| 3.3 MB 35.7 MB/s 
     |██████████████████████████

## model and tokenizer



In [4]:
model_config = {
    "num_labels": 3,
    "id2label": {0: 0, 1: 1, 2: 2},
    "label2id": {0: 0, 1: 1, 2: 2}
}

In [5]:
from transformers import ElectraForSequenceClassification, ElectraTokenizer

# model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", **model_config)
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/458 [00:00<?, ?B/s]

## datasets

In [6]:
from data_loader import get_data_loaders

train_data, val_data, test_data = get_data_loaders(tokenizer, return_loader=False)

Using custom data configuration default-86e37b266c43edeb


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-86e37b266c43edeb/0.0.0/9144e0a4e8435090117cea53e6c7537173ef2304525df4a077c435d8ee7828ff. Subsequent calls will reuse this data.


  0%|          | 0/40242 [00:00<?, ?ex/s]

  0%|          | 0/4472 [00:00<?, ?ex/s]

  0%|          | 0/5000 [00:00<?, ?ex/s]

## metric

In [7]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred, average="macro")

    return {"accuracy": accuracy, "f1": f1}

## Do Train

In [8]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from transformers import set_seed

set_seed(42)

In [10]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=f'{path_to_drive}/koelectra-small-v3-discriminator-test',
    seed=42,
    num_train_epochs=200,
    learning_rate=1e-4,
    weight_decay=0.0,
    gradient_accumulation_steps=1,
    adam_epsilon=1e-8,
    max_grad_norm=1.0,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,

    # checkpoint
    overwrite_output_dir=True,
    save_strategy='steps',
    save_steps=1000,

    # evaluation
    evaluation_strategy='steps',
    eval_steps=1000,
    metric_for_best_model="f1",

    # early stopping
    load_best_model_at_end=True
)

In [13]:
from trainer import ImbalancedSamplerTrainer

def model_init():
    return ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator", **model_config, return_dict=True)

trainer = ImbalancedSamplerTrainer(
    model_init=model_init,
    args=args,
    train_dataset=train_data,
    eval_dataset=val_data,
    compute_metrics=compute_metrics,
)

loading configuration file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/bd0f09888c5a5619ddb9de81d4a9936a94e5f45064f9a23ba6d39241ceebce02.d2485d28e5c07ca60bfa4fe84af673e0df83401e5c56bcdd991878cb4966eb34
Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "0": 0,
    "1": 1,
    "2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary

Downloading:   0%|          | 0.00/56.6M [00:00<?, ?B/s]

storing https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/052bc3ecf8c8484f1519650794b32b6d2c70750bcba71d1f763951514b5cf0c8.84bb33167d2e89d46f4cde129b2f4c447618ac33ac46f2012ee9e5e706fec112
creating metadata file for /root/.cache/huggingface/transformers/052bc3ecf8c8484f1519650794b32b6d2c70750bcba71d1f763951514b5cf0c8.84bb33167d2e89d46f4cde129b2f4c447618ac33ac46f2012ee9e5e706fec112
loading weights file https://huggingface.co/monologg/koelectra-small-v3-discriminator/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/052bc3ecf8c8484f1519650794b32b6d2c70750bcba71d1f763951514b5cf0c8.84bb33167d2e89d46f4cde129b2f4c447618ac33ac46f2012ee9e5e706fec112
Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dens

# install ray

In [12]:
!pip install "ray[tune]" transformers datasets scipy sklearn torch
!pip install hyperopt

     |████████████████████████████████| 49.6 MB 6.1 kB/s 
     |████████████████████████████████| 72 kB 527 kB/s 
     |████████████████████████████████| 124 kB 33.4 MB/s 


# haperparameter search

In [ ]:
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import ASHAScheduler

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    search_alg=HyperOptSearch(metric="objective", mode="max"),
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    scheduler=ASHAScheduler(metric="objective", mode="max")
)


No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU and 1 GPU for each trial.
/usr/local/lib/python3.7/dist-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)


== Status ==
Memory usage on this node: 2.2/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.31 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-09-06_07-22-51
Number of trials: 1/20 (1 RUNNING)
+---------------------+----------+-------+-----------------+--------------------+-------------------------------+---------+
| Trial name          | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |    seed |
|---------------------+----------+-------+-----------------+--------------------+-------------------------------+---------|
| _objective_3ed370de | RUNNING  |       |     6.75848e-05 |                  1 |                            64 | 16.9508 |
+---------------------+----------+-------+-----------------+--------------------+-------------------------------+--

(pid=445) Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
(pid=445) - This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=445) - This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=445) Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3

Result for _objective_3ed370de:
  {}
  
== Status ==
Memory usage on this node: 2.2/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: None
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.31 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_objective_2021-09-06_07-22-51
Number of trials: 3/20 (1 ERROR, 1 PENDING, 1 RUNNING)
+---------------------+----------+-------+-----------------+--------------------+-------------------------------+---------+
| Trial name          | status   | loc   |   learning_rate |   num_train_epochs |   per_device_train_batch_size |    seed |
|---------------------+----------+-------+-----------------+--------------------+-------------------------------+---------|
| _objective_3eedb502 | RUNNING  |       |     1.91443e-05 |                  3 |                            32 | 14.0759 |
| _objective_4495c1d4 | PENDING  |       |     1.49065e

(pid=444) Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias']
(pid=444) - This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(pid=444) - This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(pid=444) Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3

...

 26%|██▋       | 1000/3774 [21:19<58:52,  1.27s/it]


(pid=444) {'loss': 0.5714, 'learning_rate': 1.4071640482209623e-05, 'epoch': 0.79}


(pid=444) 
  0%|          | 0/140 [00:00<?, ?it/s]
(pid=444) 
  1%|▏         | 2/140 [00:00<00:30,  4.55it/s]
(pid=444) 
  2%|▏         | 3/140 [00:00<00:42,  3.22it/s]
(pid=444) 
  3%|▎         | 4/140 [00:01<00:48,  2.78it/s]
(pid=444) 
  4%|▎         | 5/140 [00:01<00:52,  2.58it/s]
(pid=444) 
  4%|▍         | 6/140 [00:02<00:54,  2.47it/s]
(pid=444) 
  5%|▌         | 7/140 [00:02<00:55,  2.40it/s]
(pid=444) 
  6%|▌         | 8/140 [00:03<00:56,  2.35it/s]
(pid=444) 
  6%|▋         | 9/140 [00:03<00:56,  2.33it/s]
(pid=444) 
  7%|▋         | 10/140 [00:03<00:56,  2.32it/s]
(pid=444) 
  8%|▊         | 11/140 [00:04<00:55,  2.31it/s]
(pid=444) 
  9%|▊         | 12/140 [00:04<00:55,  2.30it/s]
(pid=444) 
  9%|▉         | 13/140 [00:05<00:55,  2.30it/s]
(pid=444) 
 10%|█         | 14/140 [00:05<00:55,  2.29it/s]
(pid=444) 
 11%|█         | 15/140 [00:06<00:54,  2.28it/s]
(pid=444) 
 11%|█▏        | 16/140 [00:06<00:54,  2.27it/s]
(pid=444) 
 12%|█▏        | 17/140 [00:07<00:54,  2.28it/

Result for _objective_3eedb502:
  date: 2021-09-06_07-45-30
  done: false
  epoch: 0.79
  eval_accuracy: 0.8859570661896243
  eval_f1: 0.6863608140634808
  eval_loss: 0.31909236311912537
  eval_runtime: 61.4138
  eval_samples_per_second: 72.818
  eval_steps_per_second: 2.28
  experiment_id: bcd5e511d78d445998ad6e41c15647f2
  hostname: f7c961d7826d
  iterations_since_restore: 1
  node_ip: 172.28.0.2
  objective: 1.572317880253105
  pid: 444
  time_since_restore: 1346.2077100276947
  time_this_iter_s: 1346.2077100276947
  time_total_s: 1346.2077100276947
  timestamp: 1630914330
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 3eedb502
  
== Status ==
Memory usage on this node: 3.3/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 64.000: None | Iter 16.000: None | Iter 4.000: None | Iter 1.000: 1.572317880253105
Resources requested: 1.0/2 CPUs, 1.0/1 GPUs, 0.0/7.31 GiB heap, 0.0/3.66 GiB objects (0.0/1.0 accelerator_type:K80)
Result logdir: /root/ray_results/_obj

(pid=444) 
                                                   
100%|██████████| 140/140 [01:00<00:00,  2.44it/s]
                                                 
 31%|███       | 1163/3774 [25:51<56:03,  1.29s/it]
